In [3]:
import pandas as pd
sota_res = pd.read_csv("sota_res.csv")
sota_res_dict = {row["problem"]: (row["distance"], row["vehicle"]) for _, row in sota_res.iterrows()}
result_to_compare = [("ges_1200", "amlt/vrptw_ges_1200/vrptw_ortec/ges_res_ortec_20220708_0405.csv"),
                     ("hybrid_120", "amlt/vrptw_hybrid_120/vrptw_ortec/hybrid_res_ortec_20220706_1102.csv"),
                     ("hybrid", "amlt/vrptw_hybrid_ges_only/vrptw_ortec/hybrid_res_ortec_20220708_0407.csv")]

df = None
for name, file_name in result_to_compare:
    _df = pd.read_csv(file_name)
    _df = _df[["problem", "vehicles", 'total_cost']]
    _df = _df[_df["vehicles"] > 0]
    _df.columns = ["problem", "%s_vehicles"%name, "%s_cost"%name]
    if df is not None: df = pd.merge(df, _df)
    else: df = _df.copy(deep=True)
    
df.loc[:, "sota_vehicles"] = df["problem"].map(lambda x: sota_res_dict.get(str.lower(x.split('.')[0]), (1, 1))[1])
df.loc[:, "sota_cost"] = df["problem"].map(lambda x: sota_res_dict.get(str.lower(x.split('.')[0]), (1, 1))[0])
print(df.shape)
df.mean(axis=0)

(157, 9)


ges_1200_vehicles          16.662420
ges_1200_cost          165361.624204
hybrid_120_vehicles        16.350318
hybrid_120_cost        158992.560510
hybrid_vehicles            16.305732
hybrid_cost            158345.611465
sota_vehicles               1.000000
sota_cost                   1.000000
dtype: float64

In [6]:
import os
experiment_names = ["vrptw_ges_1200", "vrptw_hybrid_ges_only"]
instance_names = ["200", "400", "600", "800", "1000", 'ortec']

res_df_list = []

for instance in instance_names:
    df = None
    for experiment in experiment_names:
        for _, _, all_files in os.walk(f"amlt/{experiment}/vrptw_{instance}/"):
            file_name = sorted(all_files)[-1]
        _df = pd.read_csv(f"amlt/{experiment}/vrptw_{instance}/{file_name}")
        _df.loc[:, "instance"] = instance
        _df = _df[["problem", "vehicles", 'total_cost', 'instance']]
        _df = _df[_df["vehicles"] > 0]
        _df.columns = ["problem", "%s_vehicles"%experiment, "%s_cost"%experiment, 'instance']
        if df is not None: df = pd.merge(df, _df)
        else: df = _df.copy(deep=True)
    df.loc[:, "sota_vehicles"] = df["problem"].map(lambda x: sota_res_dict.get(str.lower(x.split('.')[0]), (None, None))[1])
    df.loc[:, "sota_cost"] = df["problem"].map(lambda x: sota_res_dict.get(str.lower(x.split('.')[0]), (None, None))[0])
    df.loc[:, "sota_vehicles"].fillna(df[f"{experiment_names[0]}_vehicles"], inplace=True)
    df.loc[:, "sota_cost"].fillna(df[f"{experiment_names[0]}_cost"], inplace=True)
    for experiment in experiment_names:
        df.loc[:, f'{experiment}_gap'] = df.apply(lambda x: round((x[f'{experiment}_cost']-x['sota_cost'])/x['sota_cost'], 3), axis=1)
    print(f"************{instance}**********")
    print(df.mean(axis=0))
    res_df_list.append(df)

df_all = pd.concat(res_df_list)
df_all.to_csv("res_all.csv", index=False)
print(f"************ALL**********")
print(df_all.mean(axis=0))


************200**********
vrptw_ges_1200_vehicles             13.219512
vrptw_ges_1200_cost               2648.498293
vrptw_heuristic_vehicles            17.048780
vrptw_heuristic_cost              7230.189756
vrptw_hybrid_ges_only_vehicles      13.195122
vrptw_hybrid_ges_only_cost        2624.229268
sota_vehicles                       11.926829
sota_cost                         2802.231841
vrptw_ges_1200_gap                  -0.045439
vrptw_heuristic_gap                  1.903317
vrptw_hybrid_ges_only_gap           -0.052805
dtype: float64
************400**********
vrptw_ges_1200_vehicles              25.526316
vrptw_ges_1200_cost                5443.394737
vrptw_heuristic_vehicles             33.263158
vrptw_heuristic_cost              14445.923684
vrptw_hybrid_ges_only_vehicles       25.578947
vrptw_hybrid_ges_only_cost         5423.422105
sota_vehicles                        25.263158
sota_cost                          5646.106126
vrptw_ges_1200_gap                   -0.037263
vrpt